In [2]:
import bitstring
import random
from typing import List, Tuple
import sys

In [3]:
def float_to_binary(num):
    # Round the number to 2 decimal places
    # num = round(num, 2)
    # Convert the rounded float to binary
    binary = bitstring.BitArray(float=num, length=32)
    return binary.bin

def binary_to_float(binary_str):
    # Create a BitArray from the binary string
    binary = bitstring.BitArray(bin=binary_str)
    # Convert binary to float
    return binary.float
# Example usage
num = 3.1
num2=20.2
num3=100.4
binary_representation = float_to_binary(num)
binary_representation2 = float_to_binary(num2)
binary_representation3 = float_to_binary(num3)
bn = binary_to_float(binary_representation)
print(f"Binary representation of {num} rounded to two decimals is \n{binary_representation}\n{binary_representation2}\n{binary_representation3}")



Binary representation of 3.1 rounded to two decimals is 
01000000010001100110011001100110
01000001101000011001100110011010
01000010110010001100110011001101


In [4]:
print(binary_representation[1])

1


In [22]:
from typing import List, Tuple
class GeneticEquationSolver:
  def __init__(self, step_must_run, population_size,
               elitist_rate, mutation_rate,float_decimal,func,solution_count,
               min_root,max_root,select_parent_rate,best_parent_rate):
    self.population =[]# List[Tuple[float, str]]
    self.population_size = population_size
    self.elitist_rate = elitist_rate
    self.step_must_run = step_must_run
    self.mutation_rate = mutation_rate
    self.func = func
    self.solution_count = solution_count # equall to the highest digree of variable x in polynomial
    self.min_root = min_root
    self.max_root = max_root
    self.chromosome_length = 32
    self.float_decimal =float_decimal#when generate float number must specify how many decimal want for each number
    self.select_parent_rate=select_parent_rate
    self.best_parent_rate = best_parent_rate

  def create_generation(self , count=0 ):
    if count ==0:
      count = self.population_size
    for _ in range(count):
        individual = round(random.uniform(self.min_root,self.max_root), self.float_decimal)
        binary = float_to_binary(individual)
        fitness = self.fitness(binary)
        # new = (fitness,binary)
        # self.population+=[new]
        self.population.append((fitness,binary))
    # return self.population

  def fitness(self, chromosome):
    float_number =round(binary_to_float(chromosome),3)
    return round(abs(self.func(float_number)),4)

  def select_some_best_parent(self):
    self.population = sorted(self.population, key=lambda x: x[0])
    
    best_parent_count =int(  self.best_parent_rate * self.population_size)

    if best_parent_count %2 !=0:
      best_parent_count-=1

    parent_count =int(  self.select_parent_rate * best_parent_count)
    if parent_count%2 !=0:
      parent_count-=1

    res= random.sample(range(0,best_parent_count),parent_count)

    parent_count =int(  self.select_parent_rate * self.population_size)
    if parent_count%2 !=0:
      parent_count-=1
    res+=random.sample(range(best_parent_count , self.population_size),parent_count)
    return res
  def select_random_parent(self):
    parent_count =int(  self.select_parent_rate * self.population_size)
    if parent_count%2 !=0:
      parent_count-=1
    return random.sample(range(0,self.population_size + 1),parent_count)

  def one_point_crossover(self,chr1,chr2):#chr => chromosome
    random_number = random.randint(0,self.chromosome_length)
    res1 = chr1[0:random_number]+chr2[random_number:]
    res2 = chr2[0:random_number]+chr1[random_number:]
    return [res1,res2]

  def bit_flipping_mutation(self,chr):
    bit_flip_count = random.randint(0,self.chromosome_length)
    indexes = random.sample(range(0,self.chromosome_length), bit_flip_count)
    # print(f"in bit_flipping_mutation function \n bit_flip_count:{bit_flip_count}\nindxes:{indexes}")
    for i in range(bit_flip_count):

      if chr[indexes[i]]=='0':
        chr = chr[:indexes[i]] + '1' + chr[indexes[i] + 1:]
        # chr[indexes[i]] = '1'
      else:
        chr = chr[:indexes[i]] + '0' + chr[indexes[i] + 1:]
        # chr[indexes[i]] = '0'
    return chr
  def select_some_best_new_generation(self):
    self.population = sorted(self.population, key=lambda x: x[0])
    best_parent_count =int(  self.best_parent_rate * self.population_size)

    best_gen_count = int(self.population_size * 0.1)
    self.population=self.population[:best_gen_count]
    self.population+=self.population[best_parent_count:self.population_size - best_gen_count]
  def select_random_new_generation(self):
    random.shuffle(self.population)
    # del self.population[self.population_size:]
    # print(self.population[self.population_size:])
    self.population=self.population[:self.population_size]
  def remove_some_duplicate(self):
    current =self.population[0] 
    count = 1
    for index,obj in enumerate(self.population):
      if current==obj:
        if random.randint(-1,1) == 1:
          del self.population[index]
          count+=1
    self.create_generation(count)


In [ ]:
import sys
# polynomial_func1 = lambda x: float(x**3 - 6*x**2 + 11*x - 6 ) # Roots are 1, 2, 3
polynomial_func1 = lambda x: float(4*x**2 - 190*x+14) # Roots are 1, 2, 3

GP=GeneticEquationSolver(step_must_run=100,population_size=10000,elitist_rate=0.5,mutation_rate=0.1
                         ,float_decimal=32,func=polynomial_func1,solution_count=3,min_root=-500,
                         max_root=500,select_parent_rate=0.6,best_parent_rate=0.1)#step 1&2
GP.create_generation()
for ii in range(GP.step_must_run):
    #step3
    # parent_indexes =GP.select_random_parent()#get list which has even length
    parent_indexes =GP.select_some_best_parent()#get list which has even length
    #step4&5&6
    for i in range(0, len(parent_indexes), 2):
        children = GP.one_point_crossover(GP.population[i][1],GP.population[i+1][1])
        if random.random() < GP.mutation_rate:
            children[0] =GP.bit_flipping_mutation( children[0])
        if random.random() < GP.mutation_rate:
            children[1] =GP.bit_flipping_mutation( children[1])

        fit1 = GP.fitness(children[0])
        fit2 = GP.fitness(children[1])
        GP.population.append((fit1,children[0]))
        GP.population.append((fit2,children[1]))
    # step7
    # GP.select_random_new_generation()
    GP.select_some_best_new_generation()
    sorted_data = sorted(GP.population, key=lambda x: x[0])
    # for x in range(3):
    #     print(f"generation {i} best answer is:\n{sorted_data[x][1]}:{sorted_data[x][0]}")
    # print(GP.population)
    GP.remove_some_duplicate()
    sys.stdout.write(f'\rStep {ii+1}/{GP.step_must_run}')
    sys.stdout.flush()


for x in range(20):
    print(f"best answer is:\n{sorted_data[x][1]}={round(binary_to_float(sorted_data[x][1]),3)}\nwith fitness:{sorted_data[x][0]}")
# print(GP.population)